In [1]:
import cv2
import os
import torch
from tqdm import tqdm
from PIL import Image

In [2]:
data_root = './data/dataset/test/'
images = [i for i in os.listdir(data_root)]
images.sort()

In [3]:
# --conf 0.25 --iou 0.45  # speed
# --conf 0.001 --iou 0.65  # mAP

In [4]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./runs/train/exp8/weights/best.pt')

model.conf = 0.01
model.iou = 0.5

Using cache found in /opt/ml/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-10-3 torch 1.7.1 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)

Fusing layers... 
Model Summary: 606 layers, 141082660 parameters, 0 gradients, 221.4 GFLOPs
Adding AutoShape... 


In [5]:
submission = []
for image in tqdm(images):
    image_id = image
    image = Image.open(os.path.join(data_root, image))
    
    results = model([image], size=1024)

    to_string = results.pandas().xyxy[0].drop(['name'], axis=1)
    to_string = to_string[['class', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax']].astype(str)
    to_string = to_string.values.tolist()
    to_string = ' '.join([' '.join(s) for s in to_string])
    submission.append({"PredictionString":to_string, "image_id":f"test/{image_id}"})

100%|██████████| 4871/4871 [05:45<00:00, 14.08it/s]


In [6]:
version = 'v11'
import csv
with open(f"yolov5_{version}.csv",'w') as f:
    w = csv.DictWriter(f,  fieldnames=["PredictionString","image_id"])
    w.writeheader()
    w.writerows(submission)